In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box
import re


from shapely.geometry import Point

In [12]:
%cd ..

c:\Users\javie\OneDrive\Desktop\magister\Tesis


C:\Users\javie\AppData\Roaming\Python\Python311\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### read kmz

In [13]:
# read kmz
j1_viirs = gpd.read_file(r"data\EEUU\URT\J1_VIIRS_C2_USA_contiguous_and_Hawaii_7d_1745198875185.kml")
j2_viirs = gpd.read_file(r"data\EEUU\URT\J2_VIIRS_C2_USA_contiguous_and_Hawaii_7d_1745198850787.kml")
suomi_viirs = gpd.read_file(r"data\EEUU\URT\SUOMI_VIIRS_C2_USA_contiguous_and_Hawaii_7d_1745198856426.kml")
modis = gpd.read_file(r"data\EEUU\URT\MODIS_C61_USA_contiguous_and_Hawaii_7d_1745198859398.kml")
landsat = gpd.read_file(r'data\EEUU\URT\LANDSAT_USA_contiguous_and_Hawaii_7d_1745198890398.kml')


c:\Users\javie\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in 'J1_VIIRS_C2_USA_contiguous_and_Hawaii_7d_1745198875185.kml': '375m Fire Detection Centroids (Last 0 to 6hrs) ' (default), '375m Fire Detection Centroids (Last 6 to 12hrs) ', '375m Fire Detection Centroids (Last 12 to 24hrs) ', '375m Fire Detection Centroids (older than the last 24hrs) ', '375m Fire Detection Footprints (Last 0 to 6hrs) ', '375m Fire Detection Footprints (Last 6 to 12hrs) ', '375m Fire Detection Footprints (Last 12 to 24hrs) ', '375m Fire Detection Footprints (older than the last 24hrs) '. Specify layer parameter to avoid this warning.
  result = read_func(
c:\Users\javie\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in 'J2_VIIRS_C2_USA_contiguous_and_Hawaii_7d_1745198850787.kml': '375m Fire Detection Centroids (Last 0 to 6hrs) ' (default), '375m Fire Det

In [28]:
j1_viirs['Description'][0]

'<b>Latitude: </b> 47.56953<br/>  <b>Longitude: </b> -111.04112<br/>  <b>Detection Time: </b> 2025-04-20 20:57 UTC<br/>  <b>Sensor: </b> NOAA-20 VIIRS<br/> <b>Confidence: </b> Nominal<br/>  <b>Day/Night: </b> Day<br/> <b>Scan: </b> 0.34 km<br/> <b>Track: </b> 0.56 km<br/> <b>FRP: </b> 4.13 MW<br/> <b>Brightness: </b> 346.76 K'

In [15]:
# Función para extraer las etiquetas
def parse_description(desc):
    # Usamos expresiones regulares para extraer los valores
    data = {
        'Latitude': re.search(r'Latitude:\s*</b>\s*([\d\.-]+)', desc),
        'Longitude': re.search(r'Longitude:\s*</b>\s*([\d\.-]+)', desc),
        'DetectionTime': re.search(r'Detection Time:\s*</b>\s*([\d\- :]+)', desc),
        'Sensor': re.search(r'Sensor:\s*</b>\s*([\w\- ]+)', desc),
        'Confidence': re.search(r'Confidence:\s*</b>\s*([\w]+)', desc),
        'DayNight': re.search(r'Day/Night:\s*</b>\s*([\w]+)', desc),
        'Scan_km': re.search(r'Scan:\s*</b>\s*([\d\.]+)', desc),
        'Track_km': re.search(r'Track:\s*</b>\s*([\d\.]+)', desc),
        'FRP_MW': re.search(r'FRP:\s*</b>\s*([\d\.]+)', desc),
        'Brightness_K': re.search(r'Brightness:\s*</b>\s*([\d\.]+)', desc),
    }
    # Extrae solo el grupo encontrado (group(1)) o None
    return {k: v.group(1) if v else None for k, v in data.items()}

def agrergar_cols(df):
    # Aplica la función a cada descripción
    parsed_data = df['Description'].apply(parse_description).apply(pd.Series)

    # Convierte algunos valores a tipos útiles (fechas y números)
    parsed_data['DetectionTime'] = pd.to_datetime(parsed_data['DetectionTime'], errors='coerce')
    for col in ['Latitude', 'Longitude', 'Scan_km', 'Track_km', 'FRP_MW', 'Brightness_K']:
        parsed_data[col] = pd.to_numeric(parsed_data[col], errors='coerce')

    # Une con el GeoDataFrame original
    return df.join(parsed_data)


In [ ]:
j1_viirs = agrergar_cols(j1_viirs)

j2_viirs = agrergar_cols(j2_viirs)
modis = agrergar_cols(modis)
suomi_viirs = agrergar_cols(suomi_viirs)
landsat = agrergar_cols(landsat)

In [ ]:
suomi_viirs = agrergar_cols(suomi_viirs)

### API

In [ ]:
satellite = ['VIIRS_NOAA20_NRT', 'MODIS_NRT', 'VIIRS_NOAA21_NRT', 'VIIRS_SNPP_NRT', 'LANDSAT_NRT']

In [2]:
MAP_KEY = '0b459343230aa13d73681c87891d464c'

In [ ]:
## api conection:
satellite = 'LANDSAT_NRT'


usa_url = 'https://firms.modaps.eosdis.nasa.gov/api/country/csv/' + MAP_KEY + f'/{satellite}/USA/9'
df_usa = pd.read_csv(usa_url)

In [7]:
def cambiar_fecha(df):
    df['acq_datetime'] = pd.to_datetime(df['acq_date'].astype(str) + ' ' + df['acq_time'].astype(str).str.zfill(4), format='%Y-%m-%d %H%M')
    return df

In [72]:
def guardar_datos_api(df, path_save):
    df = cambiar_fecha(df)

    df["geometry"] = df.apply(lambda row: Point(row["longitude"], row["latitude"]), axis=1)

    # Convertir a GeoDataFrame
    df = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

    print('guardando archivo')
    df.to_file(path_save, driver="ESRI Shapefile")

In [82]:
guardar_datos_api(df_usa, 'data/EEUU/URT/API/LANDSAT_2004.shp')

guardando archivo


C:\Users\javie\AppData\Local\Temp\ipykernel_6472\807974916.py:10: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  df.to_file(path_save, driver="ESRI Shapefile")
c:\Users\javie\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'acq_datetime' to 'acq_dateti'
  ogr_write(
c:\Users\javie\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field acq_dateti create as date field, though DateTime requested.
  ogr_write(


In [ ]:
# 'data/EEUU/URT/API/NOAA20_2004.shp', 'data/EEUU/URT/API/modis_2004.shp', 'data/EEUU/URT/API/NOAA21_2004.shp', 'data/EEUU/URT/API/SUOMI_2004.shp', 'data/EEUU/URT/API/LANDSAT_2004.shp'

In [4]:
area_url = 'https://firms.modaps.eosdis.nasa.gov/api/area/csv/' + MAP_KEY + '/VIIRS_NOAA20_NRT/-83.97421,35.34632,-83.91338,35.39466/3'

In [ ]:
df_area = pd.read_csv(area_url)


In [8]:
cambiar_fecha(df_area)

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,acq_datetime
0,35.35469,-83.95306,298.77,0.48,0.65,2025-04-19,631,N20,VIIRS,n,2.0NRT,285.62,0.57,N,2025-04-19 06:31:00
1,35.35818,-83.94957,296.88,0.48,0.65,2025-04-19,631,N20,VIIRS,n,2.0NRT,286.38,0.68,N,2025-04-19 06:31:00
2,35.35840,-83.95387,302.87,0.48,0.65,2025-04-19,631,N20,VIIRS,n,2.0NRT,286.00,0.68,N,2025-04-19 06:31:00
3,35.36094,-83.95815,299.34,0.48,0.65,2025-04-19,631,N20,VIIRS,n,2.0NRT,285.57,1.56,N,2025-04-19 06:31:00
4,35.36505,-83.96644,299.34,0.48,0.65,2025-04-19,631,N20,VIIRS,n,2.0NRT,284.68,0.67,N,2025-04-19 06:31:00
5,35.36732,-83.96641,301.87,0.48,0.65,2025-04-19,631,N20,VIIRS,n,2.0NRT,285.41,1.46,N,2025-04-19 06:31:00
6,35.37402,-83.96383,295.58,0.42,0.61,2025-04-19,811,N20,VIIRS,n,2.0NRT,280.50,0.57,N,2025-04-19 08:11:00
7,35.37409,-83.96257,295.28,0.42,0.61,2025-04-19,811,N20,VIIRS,n,2.0NRT,280.54,0.55,N,2025-04-19 08:11:00
8,35.35420,-83.95432,367.00,0.34,0.56,2025-04-19,1752,N20,VIIRS,h,2.0NRT,295.92,4.40,D,2025-04-19 17:52:00
9,35.35496,-83.95129,329.19,0.34,0.56,2025-04-19,1752,N20,VIIRS,n,2.0NRT,292.65,4.40,D,2025-04-19 17:52:00


### kml

In [49]:
usa_url = 'https://firms.modaps.eosdis.nasa.gov/api/kml_fire_footprints/usa_contiguous_and_hawaii/7d/landsat'

In [50]:
df_usa = gpd.read_file(usa_url)

df_usa = agrergar_cols(df_usa)
df_usa['DetectionTime'].value_counts()

c:\Users\javie\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in 'pyogrio_db92d9ffae414dc2b32ef7a3e64892d9.zip': '30m Fire Detection Centroids (Last 6 to 12hrs) ' (default), '30m Fire Detection Centroids (Last 12 to 24hrs) ', '30m Fire Detection Centroids (older than the last 24hrs) ', '30m Fire Detection Footprints (Last 6 to 12hrs) ', '30m Fire Detection Footprints (Last 12 to 24hrs) ', '30m Fire Detection Footprints (older than the last 24hrs) '. Specify layer parameter to avoid this warning.
  result = read_func(


DetectionTime
2025-04-20 17:40:00    397
2025-04-20 17:39:00    228
2025-04-20 17:37:00    117
2025-04-20 18:28:00     88
2025-04-20 15:57:00     73
2025-04-20 16:50:00     54
2025-04-20 17:34:00     47
2025-04-20 15:59:00     31
2025-04-20 16:00:00     30
2025-04-20 16:52:00     20
2025-04-20 17:36:00     16
2025-04-20 18:24:00     16
2025-04-20 15:56:00     14
2025-04-20 16:01:00     12
2025-04-20 15:58:00      8
2025-04-20 17:38:00      7
2025-04-20 18:27:00      7
2025-04-20 16:48:00      6
2025-04-20 17:35:00      2
2025-04-20 17:33:00      2
2025-04-20 18:25:00      2
Name: count, dtype: int64

In [51]:
df_usa.to_file('data/EEUU/landsat_2004_kml', driver="ESRI Shapefile")

C:\Users\javie\AppData\Local\Temp\ipykernel_20448\2338462743.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  df_usa.to_file('data/EEUU/landsat_2004_kml', driver="ESRI Shapefile")
c:\Users\javie\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Description' to 'Descriptio'
  ogr_write(
c:\Users\javie\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'DetectionTime' to 'DetectionT'
  ogr_write(
c:\Users\javie\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field DetectionT create as date field, though DateTime requested.
  ogr_write(
c:\Users\javie\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Brightness_K' to 'Brightness'
  ogr_write(
c:\Users\javie\